In [1]:
%load_ext autoreload

In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [3]:
%autoreload
from trainer import *
from SupContrastLoss import MultiPosConLoss
from RankingLoss import MultipleNegativesSymmetricRankingLoss

2024-09-13 11:26:18.037185: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-13 11:26:18.467810: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-13 11:26:19.317319: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-13 11:26:19.317384: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-13 11:26:19.317451: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to regi

In [4]:
def train_datadreamer_ta2(fold, output_folder, used_loss, luar_model_path='./rrivera1849', batch_size=128, epochs=25):
    with DataDreamer(output_folder):
        dataset = DataSource(
            "Train Data",
            data=train_data_generator,
            total_num_rows=train_num_rows,
        )
        dev_dataset = DataSource(
            "Dev Data",
            data=dev_data_generator,
            total_num_rows=dev_num_rows,
        )
    
        trainer = get_luar_trainer()(
            "LUAR Trainer",
            model_name=luar_model_path,
            peft_config=LoraConfig(),
            trust_remote_code=True,
            device='cuda:0',
            dtype="bfloat16",
            force=True,
        )

        #loss = SupConLoss if used_loss=='SupConLoss' else losses.MultipleNegativesSymmetricRankingLoss
        loss = MultiPosConLoss if used_loss=='SupConLoss' else losses.MultipleNegativesSymmetricRankingLoss

        if used_loss == 'SupConLoss':
            trainer.train_with_labeled_pairs(
                train_anchors=dataset.output["anchors"],
                train_others= dataset.output["others"],
                train_labels=dataset.output["labels"],
                validation_anchors=dev_dataset.output["anchors"],
                validation_others=dev_dataset.output["others"],
                validation_labels=dev_dataset.output["labels"],
                epochs=epochs,    
                batch_size=batch_size,
                logging_steps=0.2,
                loss=loss,
                learning_rate=0.0005,
                early_stopping_threshold=0.001,
                early_stopping_patience=5,
                accelerator_config={
                    "dispatch_batches": False,
                },
                callbacks=[EpochTrackerCallback()]
            )
        else:
            trainer.train_with_positive_pairs(
                train_anchors=dataset.output["anchors"],
                train_positives=dataset.output["positives"],
                validation_anchors=dev_dataset.output["anchors"],
                validation_positives=dev_dataset.output["positives"],
                gradient_checkpointing=True,
                epochs=epochs,    
                batch_size=batch_size,
                loss=loss,
                learning_rate=0.0005,
                early_stopping_threshold=0.001,
                early_stopping_patience=5,
                accelerator_config={
                    "dispatch_batches": False,
                },
                callbacks=[EpochTrackerCallback()]
            )

### Train original TA2 system

In [5]:
# output_path = '/mnt/swordfish-pool2/milad/datadreamer-ta2/'
# luar_model_path = '/mnt/swordfish-pool2/milad/rrivera1849'
output_path = '/burg/dsi/users/ma4608/ajay-ta2-system/output'
luar_model_path = '/burg/dsi/users/ma4608/ajay-ta2-system/training_source/rrivera1849'

In [6]:
#fold = "/mnt/swordfish-pool2/milad/hiatus-data/train-test-dev split/{split}/Official Query-candidate format/"
fold = "/burg/dsi/users/ma4608/ajay-ta2-system/training_source/data/train-test-dev-split/{split}/official-query-candidate-format/"

In [7]:
train_num_rows, train_data_generator = get_data_generator(fold, "cross_genre_all", "train", split_percent=ast.literal_eval("None"))
dev_num_rows, dev_data_generator = get_data_generator(fold, "cross_genre_all", "dev", split_percent=ast.literal_eval("None"))

In [8]:
train_datadreamer_ta2(fold, output_path + '/original_ta2_model', 'MultipleNegativesSymmetricRankingLoss', luar_model_path=luar_model_path, epochs=25)

### Train original TA2 system with Supervised Contrastive Loss:

In [9]:
# output_path = '/mnt/swordfish-pool2/milad/datadreamer-ta2/'
# luar_model_path = '/mnt/swordfish-pool2/milad/rrivera1849'
output_path = '/burg/dsi/users/ma4608/ajay-ta2-system/output'
luar_model_path = '/burg/dsi/users/ma4608/ajay-ta2-system/training_source/rrivera1849'

In [10]:
fold = "/burg/dsi/users/ma4608/ajay-ta2-system/training_source/data/train-test-dev-split/{split}/official-query-candidate-format/"

In [11]:
train_num_rows, train_data_generator = get_data_generator_for_supervised_contrastive_learning(fold, "cross_genre_all", "train", split_percent=ast.literal_eval("None"))
dev_num_rows, dev_data_generator = get_data_generator_for_supervised_contrastive_learning(fold, "cross_genre_all", "dev", split_percent=ast.literal_eval("None"))

number of authors 8896
number of authors after filtering  1651
Dataset Statistics: 1651 9761
number of authors 1351
number of authors after filtering  270
Dataset Statistics: 270 1570


In [12]:
train_datadreamer_ta2(fold, output_path + '/supcon_ta2_model', 'SupConLoss', luar_model_path=luar_model_path, batch_size=128, epochs=25)

[ 🤖 DataDreamer 💤 ] Initialized. 🚀 Dreaming to folder: /burg/dsi/users/ma4608/ajay-ta2-system/output/supcon_ta2_model
[ 🤖 DataDreamer 💤 ] Step 'Train Data' is running. ⏳
[ 🤖 DataDreamer 💤 ] Step 'Train Data' will run lazily. 🥱
[ 🤖 DataDreamer 💤 ] Step 'Dev Data' is running. ⏳
[ 🤖 DataDreamer 💤 ] Step 'Dev Data' will run lazily. 🥱
[ 🤖 DataDreamer 💤 ] Trainer 'LUAR Trainer' was previously run and saved, but was outdated. 😞
[ 🤖 DataDreamer 💤 ] Trainer 'LUAR Trainer' is running. ⏳
[ 🤖 DataDreamer 💤 ] Step 'LUAR Trainer / Tokenize Train Anchors' is running. ⏳
[ 🤖 DataDreamer 💤 ] Step 'LUAR Trainer / Tokenize Train Anchors' will run lazily. 🥱
[ 🤖 DataDreamer 💤 ] Step 'LUAR Trainer / Tokenize Train Others' is running. ⏳
[ 🤖 DataDreamer 💤 ] Step 'LUAR Trainer / Tokenize Train Others' will run lazily. 🥱
[ 🤖 DataDreamer 💤 ] Step 'LUAR Trainer / Tokenize Validation Anchors' is running. ⏳
[ 🤖 DataDreamer 💤 ] Step 'LUAR Trainer / Tokenize Validation Anchors' will run lazily. 🥱
[ 🤖 DataDreamer 💤 ] S

The repository for /burg/dsi/users/ma4608/ajay-ta2-system/training_source/rrivera1849 contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co//burg/dsi/users/ma4608/ajay-ta2-system/training_source/rrivera1849.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


[ 🤖 DataDreamer 💤 ] [LUAR Trainer (/burg/dsi/users/ma4608/ajay-ta2-system/training_source/rrivera1849)] Finished loading.
[ 🤖 DataDreamer 💤 ] [ 🤗 Accelerate] Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
[ 🤖 DataDreamer 💤 ] Step 'Dev Data' finished running lazily. 🎉
[ 🤖 DataDreamer 💤 ] Step 'LUAR Trainer / Tokenize Validation Anchors' finished running lazily. 🎉
[ 🤖 DataDreamer 💤 ] Step 'LUAR Trainer / Tokenize Validation Others' finished running lazily. 🎉
[ 🤖 DataDreamer 💤 ] [LUAR Trainer (/burg/dsi/users/ma4608/ajay-ta2-system/training_source/rrivera1849)] Eval Epoch: 0.0 -- {'loss': 4.055605411529541, 'model_preparation_time': 0.0581, 'runtime': 20.6433, 'samples_per_second': 76.054, 'steps_per_second': 0.63}
[ 🤖 DataDreamer 💤 ] Step 'Train Data' finished running lazily. 🎉
[ 🤖 DataDreamer 💤 ] Step 'LUAR Trainer / Tokenize Train Anchors' fini

OutOfMemoryError: CUDA out of memory. Tried to allocate 768.00 MiB. GPU 0 has a total capacity of 39.56 GiB of which 530.69 MiB is free. Including non-PyTorch memory, this process has 39.04 GiB memory in use. Of the allocated memory 37.21 GiB is allocated by PyTorch, and 1.33 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)